In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
# Testing load graph from json
deep_reload(src)

filename = 'graph.json'

graph = src.graph.graph_from_json(filename)

In [3]:
import json

filename = 'parameters_cec_router.json'

with open(filename, 'r') as file:

    parameters = json.load(file)

In [8]:
# Producing routing inputs
deep_reload(src)

cases = src.router.produce_routing_inputs(graph, parameters)

In [30]:
# Producing raw routes
deep_reload(src)

vehicle = 'Model 3 (Short)'
depot = 2000000000

case = cases[(vehicle, depot)]

raw_routes, success = src.router.router(
    case,
)

In [31]:
# Adding node information to routes
deep_reload(src)

routes = src.router.route_information(graph, raw_routes, parameters['route_fields'])

In [32]:
raw_routes

[[2000000000, 113002, 2000000000],
 [2000000000, 250837, 2000000000],
 [2000000000, 112931, 233384, 2000000000],
 [2000000000, 113516, 229092, 2000000000],
 [2000000000, 102006, 222803, 2000000000]]